In [1]:
#!pip install ktrain

  Using cached https://files.pythonhosted.org/packages/26/94/dbee68fd93255f7604092f9cd3c60d3aa550808099443a28a36d79cd7632/ktrain-0.18.4.tar.gz
     |████████████████████████████████| 421.8MB 32kB/s 
     |████████████████████████████████| 6.7MB 42.4MB/s 
     |████████████████████████████████| 983kB 41.1MB/s 
     |████████████████████████████████| 245kB 41.2MB/s 
     |████████████████████████████████| 778kB 36.6MB/s 
     |████████████████████████████████| 471kB 34.5MB/s 
     |████████████████████████████████| 450kB 39.3MB/s 
     |████████████████████████████████| 3.9MB 40.2MB/s 
     |████████████████████████████████| 3.0MB 33.6MB/s 
     |████████████████████████████████| 890kB 22.6MB/s 
     |████████████████████████████████| 1.1MB 37.2MB/s 
  Created wheel for ktrain: filename=ktrain-0.18.4-cp36-none-any.whl size=25253172 sha256=f8047c31f58db9bfbcf209d5fb44022007c25090cadc53462b8c08e098ebe1d0
  Stored in directory: /root/.cache/pip/wheels/73/ca/65/2bff349d7d2f2e0710d993ddb5d9a2

In [2]:
#!git clone https://github.com/laxmimerit/IMDB-Movie-Reviews-Large-Dataset-50k.git

Cloning into 'IMDB-Movie-Reviews-Large-Dataset-50k'...
remote: Enumerating objects: 10, done.
remote: Counting objects: 100% (10/10), done.
remote: Compressing objects: 100% (8/8), done.
remote: Total 10 (delta 1), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (10/10), done.


In [3]:
import pandas as pd
import numpy as np
import ktrain
from ktrain import text
import tensorflow as tf

In [4]:
data_train = pd.read_excel(r'/content/IMDB-Movie-Reviews-Large-Dataset-50k/train.xlsx', dtype='str')
data_test = pd.read_excel(r'/content/IMDB-Movie-Reviews-Large-Dataset-50k/test.xlsx', dtype='str')
print(data_train.shape, data_test.shape)

(25000, 2) (25000, 2)


In [5]:
data_train.head()

,Reviews,Sentiment
0,"When I first tuned in on this morning news, I ...",neg
1,"Mere thoughts of ""Going Overboard"" (aka ""Babes...",neg
2,Why does this movie fall WELL below standards?...,neg
3,Wow and I thought that any Steven Segal movie ...,neg
4,"The story is seen before, but that does'n matt...",neg


In [6]:
data_train.tail()

,Reviews,Sentiment
24995,Everyone plays their part pretty well in this ...,pos
24996,It happened with Assault on Prescient 13 in 20...,neg
24997,My God. This movie was awful. I can't complain...,neg
24998,"When I first popped in Happy Birthday to Me, I...",neg
24999,"So why does this show suck? Unfortunately, tha...",neg


In [7]:
data_train.sample(5)

,Reviews,Sentiment
3419,One two three four five six seven eight and ba...,neg
22985,I and a friend rented this movie. We both foun...,neg
22359,This is a great movie for all ages. Its the st...,pos
14984,The title of this film is taken from a party g...,pos
16124,I saw this movie at the theaters when I was ab...,pos


In [8]:
text.print_text_classifiers()

fasttext: a fastText-like model [http://arxiv.org/pdf/1607.01759.pdf]
logreg: logistic regression using a trainable Embedding layer
nbsvm: NBSVM model [http://www.aclweb.org/anthology/P12-2018]
bigru: Bidirectional GRU with pretrained fasttext word vectors [https://fasttext.cc/docs/en/crawl-vectors.html]
standard_gru: simple 2-layer GRU with randomly initialized embeddings
bert: Bidirectional Encoder Representations from Transformers (BERT) [https://arxiv.org/abs/1810.04805]
distilbert: distilled, smaller, and faster BERT from Hugging Face [https://arxiv.org/abs/1910.01108]


In [10]:
(train, val, preproc) = text.texts_from_df(train_df=data_train, text_column='Reviews', label_columns='Sentiment', val_df=data_test, maxlen=400, preprocess_mode = 'distilbert')

preprocessing train...
language: en
train sequence lengths:
	mean : 234
	95percentile : 598
	99percentile : 913


Is Multi-Label? False
preprocessing test...
language: en
test sequence lengths:
	mean : 234
	95percentile : 598
	99percentile : 913


In [11]:
model = text.text_classifier(name = 'distilbert', train_data=train, preproc=preproc)

Is Multi-Label? False
maxlen is 400



done.


In [12]:
lerner = ktrain.get_learner(model = model, train_data=train, val_data=val, batch_size=6)

In [13]:
lerner.fit_onecycle(lr=2e-5, epochs=2)



begin training using onecycle policy with max lr of 2e-05...
Train for 4167 steps, validate for 782 steps
Epoch 1/2
4167/4167 [==============================] - 3067s 736ms/step - loss: 0.2853 - accuracy: 0.8828 - val_loss: 0.1759 - val_accuracy: 0.9283
Epoch 2/2
4167/4167 [==============================] - 3070s 737ms/step - loss: 0.1557 - accuracy: 0.9442 - val_loss: 0.0616 - val_accuracy: 0.9836


In [15]:
predictor = ktrain.get_predictor(model=lerner.model, preproc=preproc)

In [16]:
predictor.predict(['movie was okay but actors done good job'])

['neg']

In [18]:
predictor.predict(['amazing movie but actors are poor'])

['neg']

In [21]:
predictor.predict(['actor done good job, well written script'])

['pos']

In [22]:
from google.colab import drive

In [27]:
drive.mount(r'/content/dl_models/')

Drive already mounted at /content/dl_models/; to attempt to forcibly remount, call drive.mount("/content/dl_models/", force_remount=True).


In [28]:
predictor.save(r'/content/dl_models/My Drive/dl_models/distil_bert')